In [21]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Twitter

In [22]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            #print(re.sub('[(一-龥)]','',rowdata[num]))
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
    #for j in range(len(rmhdata[i])):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
    #for j in range(len(wdata[i])):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [23]:
data =[]
for i in range(1,273):
    txt1 = open(f"명종  ({i}).txt",encoding = 'utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '명종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'명종실록'))


In [24]:
len(data)

272

In [25]:
tw = Twitter()
stopword = ["데","매우","더욱","또",'것','그','이','일','수','모두',
            '하니','때문','비록','대해','하라','어찌','우리','가지',
            '지금','반드시','다시','또한','수가','만약','감히','여러','먼저',
            '서로','여러'
           ]
noundata = []
for i in range(len(data)):
    noun = tw.nouns(''.join(data[i]))
    nn =[]
    for j in noun:#
        if j not in stopword: # 불용어 지정 
            if len(j)>1:# 단어 두글자 이상
                nn.append(j)
    noundata.append(nn)
    print(i,'번째 nouns 완료')

C:\Users\nyc15\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


0 번째 nouns 완료
1 번째 nouns 완료
2 번째 nouns 완료
3 번째 nouns 완료
4 번째 nouns 완료
5 번째 nouns 완료
6 번째 nouns 완료
7 번째 nouns 완료
8 번째 nouns 완료
9 번째 nouns 완료
10 번째 nouns 완료
11 번째 nouns 완료
12 번째 nouns 완료
13 번째 nouns 완료
14 번째 nouns 완료
15 번째 nouns 완료
16 번째 nouns 완료
17 번째 nouns 완료
18 번째 nouns 완료
19 번째 nouns 완료
20 번째 nouns 완료
21 번째 nouns 완료
22 번째 nouns 완료
23 번째 nouns 완료
24 번째 nouns 완료
25 번째 nouns 완료
26 번째 nouns 완료
27 번째 nouns 완료
28 번째 nouns 완료
29 번째 nouns 완료
30 번째 nouns 완료
31 번째 nouns 완료
32 번째 nouns 완료
33 번째 nouns 완료
34 번째 nouns 완료
35 번째 nouns 완료
36 번째 nouns 완료
37 번째 nouns 완료
38 번째 nouns 완료
39 번째 nouns 완료
40 번째 nouns 완료
41 번째 nouns 완료
42 번째 nouns 완료
43 번째 nouns 완료
44 번째 nouns 완료
45 번째 nouns 완료
46 번째 nouns 완료
47 번째 nouns 완료
48 번째 nouns 완료
49 번째 nouns 완료
50 번째 nouns 완료
51 번째 nouns 완료
52 번째 nouns 완료
53 번째 nouns 완료
54 번째 nouns 완료
55 번째 nouns 완료
56 번째 nouns 완료
57 번째 nouns 완료
58 번째 nouns 완료
59 번째 nouns 완료
60 번째 nouns 완료
61 번째 nouns 완료
62 번째 nouns 완료
63 번째 nouns 완료
64 번째 nouns 완료
65 번째 nouns 완료
66 번째 nouns 완료
67 번째

In [26]:
from gensim import corpora
dictionary = corpora.Dictionary(noundata)
corpus = [dictionary.doc2bow(text) for text in noundata]


In [27]:
import gensim
NUM_TOPICS = 20 #20 개 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.020*"사람" + 0.013*"백성" + 0.010*"나라" + 0.006*"의논"')
(1, '0.020*"사람" + 0.019*"윤임" + 0.017*"생각" + 0.013*"대비"')
(2, '0.020*"백성" + 0.012*"도적" + 0.009*"나라" + 0.008*"사람"')
(3, '0.013*"백성" + 0.009*"유성" + 0.007*"사람" + 0.006*"길이"')
(4, '0.017*"마음" + 0.017*"사람" + 0.013*"전하" + 0.011*"백성"')
(5, '0.014*"백성" + 0.009*"사람" + 0.008*"구제" + 0.006*"서원"')
(6, '0.021*"백성" + 0.015*"사람" + 0.011*"나라" + 0.007*"수령"')
(7, '0.006*"임금" + 0.006*"가요" + 0.005*"마음" + 0.005*"나라"')
(8, '0.013*"백성" + 0.012*"사람" + 0.008*"고을" + 0.006*"폐단"')
(9, '0.015*"사람" + 0.015*"임금" + 0.008*"학문" + 0.008*"혈구"')
(10, '0.014*"사람" + 0.013*"백성" + 0.009*"복창" + 0.009*"마음"')
(11, '0.021*"사람" + 0.018*"백성" + 0.008*"마음" + 0.008*"전하"')
(12, '0.027*"백성" + 0.015*"사람" + 0.008*"전교" + 0.007*"마음"')
(13, '0.020*"사람" + 0.012*"백성" + 0.007*"나라" + 0.006*"마음"')
(14, '0.030*"백성" + 0.012*"사람" + 0.010*"감사" + 0.009*"수령"')
(15, '0.020*"백성" + 0.009*"전교" + 0.009*"나라" + 0.008*"의논"')
(16, '0.024*"백성" + 0.011*"사람" + 0.009*"전교" + 0.007*"폐단"')
(17, '0.017*"백성" + 0.013

In [28]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)